In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.tokenize import word_tokenize
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

In [2]:
df = pd.read_csv("final_data.csv")
df

,label,Hindi
0,ham,विषय: एनरॉन मेथनॉल; मीटर # : 988291\nयह उस नोट...
1,ham,विषय: 9 जनवरी 2001 के लिए एचपीएल नामांकन\n(संल...
2,ham,"विषय: नियॉन रिट्रीट\nहो हो हो, हम वर्ष के उस स..."
3,spam,"विषय: फ़ोटोशॉप, विंडोज़, कार्यालय। सस्ता । मुख..."
4,ham,विषय: पुन: भारतीय स्प्रिंग्स\nयह डील टेको पीवी...
...,...,...
16612,spam,यह दूसरी बार है जब हमने आपसे संपर्क करने का प्...
16613,ham,क्या आप घर से एस्प्लेनेड जाएँगे?
16614,ham,"अफ़सोस, *उसी के मूड में था। तो...कोई अन्य सुझाव?"
16615,ham,"उस आदमी ने कुछ शरारतें कीं, लेकिन मैंने ऐसे व्..."


In [3]:
lbl = LabelEncoder()
label_encoder = lbl.fit_transform(df['label'])

In [4]:
x = df['Hindi']

In [5]:
def hindi_tokenizer(text):
    tokens = nltk.word_tokenize(text,language='hindi',preserve_line=True)

    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))

    # remove punctuation from each word
    words = [re_punc.sub('',w) for w in tokens]
    return words

In [9]:
def hindi_tokenizer(text):
    tokens = nltk.word_tokenize(text,language='hindi',preserve_line=True)

    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))

    # remove punctuation from each word
    words = [re_punc.sub('',w) for w in tokens]
    return words

In [10]:
tfidf = TfidfVectorizer(tokenizer=hindi_tokenizer)
tfidf_matrix = tfidf.fit_transform(x)

In [11]:
smote = SMOTE(random_state=0)

In [13]:
tfidf_matrix.shape,label_encoder.shape

((16617, 213106), (16617,))

In [15]:
x_sm, y_sm = smote.fit_resample(tfidf_matrix,label_encoder)

In [16]:
x_sm.shape, y_sm.shape

((24902, 213106), (24902,))

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_sm,y_sm,test_size=0.3,random_state=24)

In [18]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((17431, 213106), (17431,), (7471, 213106), (7471,))

In [19]:
svm = SVC(kernel='linear',random_state=0)

In [20]:
svm.fit(x_train,y_train)

SVC(kernel='linear', random_state=0)

In [21]:
y_pred = svm.predict(x_test)

In [22]:
accuracy_score(y_test,y_pred)

0.9856779547583991

In [23]:
f1_score(y_test,y_pred)

0.9855229332972534

In [24]:
import pickle

In [25]:
import pickle

# Assuming svm, tfidf, and label_encoder are already defined
with open("svm.pkl", "wb") as svm_out:
    pickle.dump(svm, svm_out)

with open("tfidf.pkl", "wb") as tfidf_out:
    pickle.dump(tfidf, tfidf_out)

with open("lbl.pkl", "wb") as label_encoder_out:
    pickle.dump(lbl, label_encoder_out)
